In [1]:
%cd ..

/Users/mateoibarguen/Desktop/CSC 482/complementary_fact_finder


In [56]:
import pickle
import re
import json
from src.topic_models import *

### Steps:
- Reload wikipedia corpus such that it outputs:
    - Article title: First paragraph.
- Save tokens.
- Retrain so that it finds the most informative articles. 
- Score the sentences within each article. 
- provide the most informative sentence. 

In [3]:
with open('data/articles.pkl', 'rb') as f:
    articles = pickle.load(f)

In [46]:
with open('data/tai-documents-v3.json', 'r') as f:
    students = json.load(f)

In [49]:
text = [s['plaintext'] for s in students]

In [80]:
print(text[38])

Space
Annette Salgado #24

Table of Contents
Introduction
Facts about space
Different planets
Sun
conclusion

Introduction

Do you LOVE space? Well if you do you will maybe like this book, but anyway I am going to teach you some facts about space, but I will also teach you about the different planets out there in space, but what about the sun? Well I will also teach you about the great big star in are solar system. Well what are you waiting fo go and read my book about space.

Fun facts about space

Have you heard of the black hole that people took a picture from? Well if you have how cool was it to see? But anyway here are some fun fact about space. Did you know that there are hotter planets then the sun? Well if not then the hotter or bigger sun is KELT-9b this plant is hotter than most stars it was found in the year 2016. Did you know that no one knows how old saturn’s rings are? Do you know that venus is named after the goddess of love? Her name was Venus too!

Different planets

H

In [57]:
lda_wiki = LdaWikiModel(student_tokens_path='data/student_keywords.json', wiki_tokens_path='data/wiki_keywords.pkl', wiki_sents_path='data/sents.pkl')

In [58]:
lda_wiki.predict_lda_index(30)

universe introduction know universe formed keep reading .and subtopics know universe expanding know black hole form know star form know travel time … know universe expanding everyone world know light thing universe people know travel sd say travel km sd light travel sd travel speed light vacuum mile second kilometer second notice talk heading mabe light travel universe year go universe travel universe expands space expands universe expands know black hole form imagine room lot lottery shirt fit room try turn black hole way black hole form way star burn progress start star core bit iron build core iron star dy progress called supernova leftover star iron become black hole… know hoe star form star form lot hydrogen gas cloud gravity put pursue star form know way star form star way star burn progress start star core bit iron build core iron star dy progress called supernova leftover star iron becomes black hole neutron star .these con form star… know travel time approach speed light clock

[(14354, 0.64471924),
 (8816, 0.58561075),
 (6608, 0.5851321),
 (10803, 0.57787514),
 (3281, 0.57772756),
 (1326, 0.57595545),
 (7710, 0.57368046),
 (2219, 0.57066),
 (4387, 0.56003),
 (8222, 0.5600295)]

In [ ]:
clean_text = TextCleaner()
def tfidf_score_sent(sent):
    return np.sum([t[1] for t in self.wiki_tfidf[lsi_articles.wiki_dict.doc2bow(self.clean_text.get_keywords_from_sentence(sent))]])

In [ ]:
def get_most_informative_sent(article):
    article_sents = sent_tokenize(article)
    tfidf_scores = np.array([self.tfidf_score_sent(s) for s in tfidf_score_sent(s)])
    return article_sents[np.argmax(tfidf_scores)]